### Load libraries and functions

In [1]:

import sys
sys.path.insert(0, '../src/')

from loopless_utils import loops_enumeration_from_fva

from load_modify_sample_utils import load_model, get_objective_functions, get_reaction_bounds, modify_model
from load_modify_sample_utils import sample_optgp

from correlations_utils import correlated_reactions, split_forward_reverse
from correlations_utils import find_reactants_products

from graphs_utils import construct_graph, plot_graph, compute_nodes_centrality_metrics
from graphs_utils import compare_network_modularities, compare_node_centralities, compare_essential_to_network_central_reactions

from pathways_utils import dictionary_reaction_id_to_kegg_id, get_kegg_pathways_from_reaction_ids
from pathways_utils import map_model_to_kegg_reactions_dictionary, read_json_file, fill_missing_kegg_ids_in_initial_dictionary
from pathways_utils import subset_model_reactions_from_pathway_info, subset_sampling_array_from_reaction_ids, sort_reactions_by_model_order
from pathways_utils import dictionary_reaction_id_to_pathway, dictionary_map_reverse_reaction_id_to_pathway


### Load and inspect model (for more info see `load_modify_samply.ipynb`)

In [2]:

ec_cobra_model, ec_cobra_reactions, ec_cobra_reaction_ids,  = load_model("../ext_data/models/e_coli_core.xml")

objective_functions = get_objective_functions(ec_cobra_model)
print(objective_functions)

default_reaction_bounds = get_reaction_bounds(ec_cobra_model)
print(default_reaction_bounds)


Set parameter Username
Set parameter LicenseID to value 2642044
Academic license - for non-commercial use only - expires 2026-03-25
['BIOMASS_Ecoli_core_w_GAM']
{'PFK': (0.0, 1000.0), 'PFL': (0.0, 1000.0), 'PGI': (-1000.0, 1000.0), 'PGK': (-1000.0, 1000.0), 'PGL': (0.0, 1000.0), 'ACALD': (-1000.0, 1000.0), 'AKGt2r': (-1000.0, 1000.0), 'PGM': (-1000.0, 1000.0), 'PIt2r': (-1000.0, 1000.0), 'ALCD2x': (-1000.0, 1000.0), 'ACALDt': (-1000.0, 1000.0), 'ACKr': (-1000.0, 1000.0), 'PPC': (0.0, 1000.0), 'ACONTa': (-1000.0, 1000.0), 'ACONTb': (-1000.0, 1000.0), 'ATPM': (8.39, 1000.0), 'PPCK': (0.0, 1000.0), 'ACt2r': (-1000.0, 1000.0), 'PPS': (0.0, 1000.0), 'ADK1': (-1000.0, 1000.0), 'AKGDH': (0.0, 1000.0), 'ATPS4r': (-1000.0, 1000.0), 'PTAr': (-1000.0, 1000.0), 'PYK': (0.0, 1000.0), 'BIOMASS_Ecoli_core_w_GAM': (0.0, 1000.0), 'PYRt2': (-1000.0, 1000.0), 'CO2t': (-1000.0, 1000.0), 'RPE': (-1000.0, 1000.0), 'CS': (0.0, 1000.0), 'RPI': (-1000.0, 1000.0), 'SUCCt2_2': (0.0, 1000.0), 'CYTBD': (0.0, 1000.

### Modify the model to create two different conditions (for more info see `load_modify_samply.ipynb`)

In [3]:

# Set optimal percentage to 100
ec_cobra_model_condition_100 = modify_model(
    cobra_model         = ec_cobra_model,
    objective_function  = "BIOMASS_Ecoli_core_w_GAM",
    optimal_percentage  = 100,
    objective_direction = "max"
)

updated_objective_functions = get_objective_functions(ec_cobra_model_condition_100)
print(updated_objective_functions)

updated_reaction_bounds = get_reaction_bounds(ec_cobra_model_condition_100)
print(updated_reaction_bounds.get("BIOMASS_Ecoli_core_w_GAM"))

# -----------

# Set optimal percentage to 0
ec_cobra_model_condition_0 = modify_model(
    cobra_model         = ec_cobra_model,
    objective_function  = "BIOMASS_Ecoli_core_w_GAM",
    optimal_percentage  = 0,
    objective_direction = "max"
)

updated_objective_functions = get_objective_functions(ec_cobra_model_condition_0)
print(updated_objective_functions)

updated_reaction_bounds = get_reaction_bounds(ec_cobra_model_condition_0)
print(updated_reaction_bounds.get("BIOMASS_Ecoli_core_w_GAM"))


Read LP format model from file /tmp/tmp6w5vef90.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
['BIOMASS_Ecoli_core_w_GAM']
(0.872922, 1000)
Read LP format model from file /tmp/tmp_ufhbs0f.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros
['BIOMASS_Ecoli_core_w_GAM']
(0.0, 1000)


### Identify loopy reactions in model (for more info see `loopless.ipynb`)

In [4]:

loopy_reactions_fva_100 = loops_enumeration_from_fva(ec_cobra_model_condition_100, fraction_of_optimum=0.999)
print(loopy_reactions_fva_100)

loopy_reactions_100 = [item[0] for item in loopy_reactions_fva_100]
print(loopy_reactions_100)


loopy_reactions_fva_0 = loops_enumeration_from_fva(ec_cobra_model_condition_0, fraction_of_optimum=0)
print(loopy_reactions_fva_0)

loopy_reactions_0 = [item[0] for item in loopy_reactions_fva_0]
print(loopy_reactions_0)


[('SUCDi', 994.7794007141792), ('FRD7', 995.0539767141795)]
['SUCDi', 'FRD7']
[('SUCDi', 980.0), ('FRD7', 1000.0)]
['SUCDi', 'FRD7']


### Remove loopy reactions from the 2 models created above to reduce the thermodynamically infeasible solutions from sampling

In [5]:

ec_cobra_model_condition_100.reactions.get_by_id("FRD7").bounds = (0, 0)
ec_cobra_model_condition_0.reactions.get_by_id("FRD7").bounds = (0, 0)


### Perform sampling on the modified models with the loopy reaction "FRD7" removed. (for more info see `load_modify_samply.ipynb`)

In [6]:

samples_optgp_condition_100 = sample_optgp(ec_cobra_model_condition_100, 
                                           n_samples = 3000, 
                                           thinning=100, 
                                           reaction_in_rows = True)

samples_optgp_condition_0 = sample_optgp(ec_cobra_model_condition_0, 
                                         n_samples = 3000, 
                                         thinning=100, 
                                         reaction_in_rows = True)


Read LP format model from file /tmp/tmpwvggjaa9.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros


Read LP format model from file /tmp/tmpcmxpr2e2.lp
Reading time = 0.00 seconds
: 72 rows, 190 columns, 720 nonzeros


### Map KEGG pathway information to reactions BiGG ids

Functions that are used to map information from KEGG pathways to the model reactions (BiGG/SEED ids). For more info see `pathways.ipynb`

In [7]:

initial_bigg_to_kegg_dictionary = map_model_to_kegg_reactions_dictionary(ec_cobra_model)

reactions_json, reactions_pandas = read_json_file("../ext_data/reactions/reactions.json")

bigg_to_kegg, seed_to_kegg = dictionary_reaction_id_to_kegg_id(reactions_pandas)

final_bigg_to_kegg_dictionary = fill_missing_kegg_ids_in_initial_dictionary(initial_bigg_to_kegg_dictionary, 
                                                                            modeltype="BiGG", 
                                                                            bigg_to_kegg=bigg_to_kegg,
                                                                            seed_to_kegg=seed_to_kegg)

df_kegg_pathways = get_kegg_pathways_from_reaction_ids(final_bigg_to_kegg_dictionary)


### First, we need to create the graph from the full netwrok, so we are going to split all model's reactions into forward and reverse reactions. For more info see: `correlations.ipynb`

In [8]:

extended_steady_states_100_full, extended_reactions_100 = split_forward_reverse(samples_optgp_condition_100, reactions=ec_cobra_reaction_ids)

reversibility_list_all_reactions_full, reactants_list_all_reactions_full, products_list_all_reactions_full = find_reactants_products(ec_cobra_model, extended_reactions_100)
print(len(reversibility_list_all_reactions_full), len(reactants_list_all_reactions_full))


extended_steady_states_0_full, extended_reactions_0 = split_forward_reverse(samples_optgp_condition_0, reactions=ec_cobra_reaction_ids)

reversibility_list_all_reactions_full, reactants_list_all_reactions_full, products_list_all_reactions_full = find_reactants_products(ec_cobra_model, extended_reactions_0)
print(len(reversibility_list_all_reactions_full), len(reactants_list_all_reactions_full))


118 118
126 126


### Calculate the correlation matrix of the full network (without sharing metabolites filtering). For more info see: `correlations.ipynb`

In [ ]:

(linear_correlation_matrix_100_full, 
non_linear_correlation_matrix_100_full, 
mixed_correlation_matrix_100_full, 
correlations_dictionary_100_full) = correlated_reactions(
        steady_states = extended_steady_states_100_full,
        boolean_sharing_metabolites_matrix=None,
        reactions=extended_reactions_100,
        linear_coeff = "pearson", 
        linear_corr_cutoff = 0.3, 
        indicator_cutoff = 1.2,
        jensenshannon_cutoff = 0.05,
        std_cutoff= 1e-2,
        include_non_linear = True, 
        cells = 5, 
        cop_coeff = 0.2, 
        lower_triangle = False, 
        verbose = True)



(linear_correlation_matrix_0_full, 
non_linear_correlation_matrix_0_full, 
mixed_correlation_matrix_0_full, 
correlations_dictionary_0_full) = correlated_reactions(
        steady_states = extended_steady_states_0_full,
        boolean_sharing_metabolites_matrix=None,
        reactions=extended_reactions_0,
        linear_coeff = "pearson", 
        linear_corr_cutoff = 0.3, 
        indicator_cutoff = 1.2,
        jensenshannon_cutoff = 0.05,
        std_cutoff= 1e-2,
        include_non_linear = True, 
        cells = 5, 
        cop_coeff = 0.2, 
        lower_triangle = False, 
        verbose = True)



/home/touliopoulos/miniconda3/envs/dingo-stats-3-10/lib/python3.10/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/touliopoulos/miniconda3/envs/dingo-stats-3-10/lib/python3.10/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


Completed the process of 345 from 5995 copulas
Completed the process of 378 from 5995 copulas
Completed the process of 391 from 5995 copulas
Completed the process of 392 from 5995 copulas
Completed the process of 399 from 5995 copulas
Completed the process of 607 from 5995 copulas
Completed the process of 661 from 5995 copulas
Completed the process of 663 from 5995 copulas
Completed the process of 664 from 5995 copulas
Completed the process of 665 from 5995 copulas
Completed the process of 666 from 5995 copulas
Completed the process of 671 from 5995 copulas
Completed the process of 672 from 5995 copulas
Completed the process of 680 from 5995 copulas
Completed the process of 681 from 5995 copulas
Completed the process of 682 from 5995 copulas
Completed the process of 689 from 5995 copulas
Completed the process of 690 from 5995 copulas
Completed the process of 692 from 5995 copulas
Completed the process of 698 from 5995 copulas
Completed the process of 699 from 5995 copulas
Completed the

### Create a dictionary that maps reaction ids to pathways from KEGG information and also create 2 dictionaries (one for each condition of the full network) additionally including the mapping of the reverse reactions. For more info see: `clustering.ipynb`

In [10]:

Glycolysis = subset_model_reactions_from_pathway_info(df_kegg_pathways, "Glycolysis / Gluconeogenesis")
PPP = subset_model_reactions_from_pathway_info(df_kegg_pathways, "Pentose phosphate pathway")

bigg_to_pathway_dict = dictionary_reaction_id_to_pathway(Glycolysis = Glycolysis, PPP = PPP)

group_map_100_full = dictionary_map_reverse_reaction_id_to_pathway(bigg_to_pathway_dict, extended_reactions_100)
group_map_0_full = dictionary_map_reverse_reaction_id_to_pathway(bigg_to_pathway_dict, extended_reactions_0)


### Create the graphs from the 2 conditions (full network, not subset)

The `construct_graph` function creates a networkx graph from a linear correlation matrix or from both a linear correlation and a non-linear copula dependencies matrix.  In this graph reactions are nodes and correlation values are edges. Users can also provide reaction-pathway mapping information in the `group_map` dictionary parameter and this will be saved in the graph for potential vizualization.

In [11]:

G100_full, pos100_full = construct_graph(linear_correlation_matrix_100_full,
                         non_linear_correlation_matrix_100_full,
                         reactions=extended_reactions_100,
                         remove_unconnected_nodes=False,
                         correction=False,
                         group_map=group_map_100_full)


G0_full, pos0_full = construct_graph(linear_correlation_matrix_0_full,
                         non_linear_correlation_matrix_0_full,
                         reactions=extended_reactions_0,
                         remove_unconnected_nodes=False,
                         correction=False,
                         group_map=group_map_0_full)


### Network Modularity/Centrality metrics derived from graph analysis

The `compute_nodes_centrality_metrics` function computes centrality measures for nodes (reactions) in the graph network:
- Weighted degree centrality (normalized by number of nodes)
- Betweenness centrality
- Clustering coefficient

In [12]:

centrality_dict_100 = compute_nodes_centrality_metrics(G100_full)

centrality_dict_0 = compute_nodes_centrality_metrics(G0_full)


In [13]:

print(centrality_dict_100.get("betweenness"))
print(centrality_dict_100.get("degree"))
print(centrality_dict_100.get("clustering"))

print(centrality_dict_0.get("betweenness"))
print(centrality_dict_0.get("degree"))
print(centrality_dict_0.get("clustering"))


{'PFK': 0.00809659410427305, 'PFL': 0.009691969639703589, 'PGI': 0.036490423643450294, 'PGK_rev': 0.027703712197480815, 'PGK': 0.010756575293313572, 'PGL': 0.12226800885753701, 'ACALD_rev': 0.03165496630176372, 'ACALD': 0.01887822300221725, 'AKGt2r_rev': 0.00019648295510364474, 'AKGt2r': 9.824147755182237e-05, 'PGM_rev': 0.0323880039451299, 'PGM': 0.01883818706707796, 'PIt2r': 0.0002031403446080723, 'ALCD2x_rev': 0.18451291957214105, 'ALCD2x': 0.09330560048699421, 'ACALDt_rev': 0.03228109620877644, 'ACALDt': 0.012464277319729874, 'ACKr_rev': 0.33233153263279164, 'ACKr': 0.17757103973225288, 'PPC': 0.0007269869338834856, 'ACONTa': 0.058063033939008135, 'ACONTb': 0.022995402031859177, 'ATPM': 0.0, 'PPCK': 0.0, 'ACt2r_rev': 0.09185878277120815, 'ACt2r': 0.044680584110705475, 'PPS': 0.0, 'ADK1': 0.0, 'AKGDH': 0.035545771126702944, 'ATPS4r': 0.0296266609898971, 'PTAr': 0.018847401333876256, 'PYK': 0.007756263875536856, 'BIOMASS_Ecoli_core_w_GAM': 0.0, 'PYRt2_rev': 0.00019648295510364474, 'P

### Plot the graphs from the full networks of the 2 conditions

The `plot_graph` function plots a correlation-based networkx graph with multiple visual features including:
- Node annotations (proivded in the `centralities` dict)
- Edge styles (linear or non-linear based on the provided matrices)
- Clique-based shadowing (`remove_clique_edges`, `include_matrix2_in_cliques`, `min_clique_size`, `shadow_edges`).

In [26]:

#import plotly.io as pio
#pio.renderers.default = "browser"
#pio.renderers.default = "notebook_connected"

plot_graph(G100_full, 
           pos100_full, 
           remove_clique_edges=True, 
           include_matrix2_in_cliques=False, 
           min_clique_size=10, 
           shadow_edges="mixed", 
           centralities=centrality_dict_100)


plot_graph(G0_full, 
           pos0_full, 
           remove_clique_edges=True, 
           include_matrix2_in_cliques=False, 
           min_clique_size=10, 
           shadow_edges="mixed",
           centralities=centrality_dict_0)


### Compare the 2 graph networks with modularity and node centrality metrics

The `compare_network_modularities` function compares the modularity scores of two graphs using the greedy modularity community detection method and returns the difference substracting the modularity score of graph 2 (second argument) from graph 1 (first argument). If score is positive, it means that graph 2 has a higher modularity score

The `compare_node_centralities` function compares node centralities between two centrality dictionaries for shared nodes and returns the difference by substracting the corresponding metric score of graph 2 (second argument) from graph 1 (first argument). If a score is positive, it means that graph 2 has a higher value in that score.

In [16]:


modularities_diff = compare_network_modularities(G100_full, G0_full)
print(modularities_diff)


sorted_betwenness_nodes = compare_node_centralities(centrality_dict_100.get("betweenness"), centrality_dict_0.get("betweenness"))
print(sorted_betwenness_nodes)


sorted_weighted_degree_nodes = compare_node_centralities(centrality_dict_100.get("degree"), centrality_dict_0.get("degree"))
print(sorted_weighted_degree_nodes)


sorted_clustering_coeff_nodes = compare_node_centralities(centrality_dict_100.get("degree"), centrality_dict_0.get("clustering"))
print(sorted_clustering_coeff_nodes)



-0.055669121176906144
[('D_LACt2_rev', 0.19828897636637774), ('CYTBD', 0.1663975459581592), ('ACONTa', 0.11209921049333638), ('EX_o2_e_rev', 0.10044213049838159), ('CO2t_rev', 0.09925524920326885), ('D_LACt2', 0.09238115953418442), ('EX_o2_e', 0.0886599764808096), ('BIOMASS_Ecoli_core_w_GAM', 0.07518770766680612), ('TPI', 0.0697761743661065), ('EX_nh4_e_rev', 0.06088682982032621), ('EX_lac__D_e', 0.04993344211904614), ('SUCDi', 0.049663385012404276), ('ACONTb', 0.04778307685758365), ('TKT1', 0.03875261862522035), ('AKGt2r_rev', 0.03805330098980197), ('CO2t', 0.03561044370025009), ('EX_glc__D_e_rev', 0.03542772736895446), ('PYRt2_rev', 0.034632783298112875), ('PIt2r', 0.03280010850196824), ('EX_glu__L_e', 0.03230913423840848), ('EX_nh4_e', 0.03155836144470647), ('PGK_rev', 0.0314606438436377), ('PPS', 0.020107980606913257), ('ETOHt2r_rev', 0.019501093070849275), ('PYRt2', 0.017348028290021657), ('EX_glc__D_e', 0.01715476076993903), ('NADTRHD', 0.016585728267787148), ('LDH_D_rev', 0.0161

### From now on to better illustrate functions related to graphs, we will focus only in certain subsystems of the model (selected pathways of interest). 

We will subset the sampling array to include only reactions from Glycolysis, Glyconeogenesis and the Pentose phosphate pathway. For more information on how these functions work see: `load_modify_samply.ipynb`

In [17]:

Glycolysis = subset_model_reactions_from_pathway_info(df_kegg_pathways, "Glycolysis / Gluconeogenesis")
PPP = subset_model_reactions_from_pathway_info(df_kegg_pathways, "Pentose phosphate pathway")

reactions_in_pathways_ordered_duplicates = sort_reactions_by_model_order(ec_cobra_reaction_ids, Glycolysis, PPP)


# Remove duplicates
reactions_in_pathways_ordered = []
[reactions_in_pathways_ordered.append(val) for val in reactions_in_pathways_ordered_duplicates if val not in reactions_in_pathways_ordered]


subset_pathways_optgp_condition_100 = subset_sampling_array_from_reaction_ids(
                                                                samples_optgp_condition_100, 
                                                                ec_cobra_reaction_ids, 
                                                                subset_reactions = reactions_in_pathways_ordered)


subset_pathways_optgp_condition_0 = subset_sampling_array_from_reaction_ids(
                                                                samples_optgp_condition_0, 
                                                                ec_cobra_reaction_ids, 
                                                                subset_reactions = reactions_in_pathways_ordered)


### Split bidirectional-reversible reactions (having both positive and negative flux values) into two separate reactions (forward and reverse). For more info see: `correlations.ipynb`

In [18]:


(subset_extended_steady_states_100,
 subset_extended_reactions_100) = split_forward_reverse(subset_pathways_optgp_condition_100, 
                                                        reactions=reactions_in_pathways_ordered)

(reversibility_list_all_reactions_100, 
 reactants_list_all_reactions_100,
 products_list_all_reactions_100) = find_reactants_products(ec_cobra_model, 
                                                            subset_extended_reactions_100)

print(len(reversibility_list_all_reactions_100), len(reactants_list_all_reactions_100))



(subset_extended_steady_states_0,
 subset_extended_reactions_0) = split_forward_reverse(subset_pathways_optgp_condition_0,
                                               reactions=reactions_in_pathways_ordered)

(reversibility_list_all_reactions_0,
 reactants_list_all_reactions_0,
 products_list_all_reactions_0) = find_reactants_products(ec_cobra_model, 
                                                          subset_extended_reactions_0)
 
print(len(reversibility_list_all_reactions_0), len(reactants_list_all_reactions_0))



21 21
25 25


### Calculate the correlation matrix of the 2 pathways only (without sharing metabolites filtering)

In [ ]:

(subset_linear_correlation_matrix_100, 
subset_non_linear_correlation_matrix_100, 
subset_mixed_correlation_matrix_100, 
subset_correlations_dictionary_100) = correlated_reactions(
        steady_states = subset_extended_steady_states_100,
        boolean_sharing_metabolites_matrix=None,
        reactions=subset_extended_reactions_100,
        linear_coeff = "pearson",
        linear_corr_cutoff = 0.3, 
        indicator_cutoff = 1.2,
        jensenshannon_cutoff = 0.05,
        std_cutoff= 1e-2,
        include_non_linear = True, 
        cells = 5, 
        cop_coeff = 0.2, 
        lower_triangle = False, 
        verbose = True
)


(subset_linear_correlation_matrix_0, 
subset_non_linear_correlation_matrix_0, 
subset_mixed_correlation_matrix_0, 
subset_correlations_dictionary_0) = correlated_reactions(
        steady_states = subset_extended_steady_states_0,
        boolean_sharing_metabolites_matrix=None,
        reactions=subset_extended_reactions_0,
        linear_coeff = "pearson",
        linear_corr_cutoff = 0.3, 
        indicator_cutoff = 1.2,
        jensenshannon_cutoff = 0.05,
        std_cutoff= 1e-2,
        include_non_linear = True, 
        cells = 5, 
        cop_coeff = 0.2, 
        lower_triangle = False, 
        verbose = True
)


Completed the process of 30 from 89 copulas
Completed the process of 1 from 144 copulas
Completed the process of 2 from 144 copulas
Completed the process of 3 from 144 copulas
Completed the process of 4 from 144 copulas
Completed the process of 5 from 144 copulas
Completed the process of 6 from 144 copulas
Completed the process of 7 from 144 copulas
Completed the process of 8 from 144 copulas
Completed the process of 9 from 144 copulas
Completed the process of 10 from 144 copulas
Completed the process of 11 from 144 copulas
Completed the process of 12 from 144 copulas
Completed the process of 13 from 144 copulas
Completed the process of 14 from 144 copulas
Completed the process of 15 from 144 copulas
Completed the process of 16 from 144 copulas
Completed the process of 17 from 144 copulas
Completed the process of 18 from 144 copulas
Completed the process of 19 from 144 copulas
Completed the process of 20 from 144 copulas
Completed the process of 21 from 144 copulas
Completed the proces

### Create a dictionary that maps reaction ids to pathways from KEGG information and also create 2 dictionaries (one for each condition of the full network) additionally including the mapping of the reverse reactions. For more info see: `correlations.ipynb`

In [20]:

Glycolysis = subset_model_reactions_from_pathway_info(df_kegg_pathways, "Glycolysis / Gluconeogenesis")
PPP = subset_model_reactions_from_pathway_info(df_kegg_pathways, "Pentose phosphate pathway")

bigg_to_pathway_dict = dictionary_reaction_id_to_pathway(Glycolysis = Glycolysis, PPP = PPP)

group_map_100 = dictionary_map_reverse_reaction_id_to_pathway(bigg_to_pathway_dict, subset_extended_reactions_100)
group_map_0 = dictionary_map_reverse_reaction_id_to_pathway(bigg_to_pathway_dict, subset_extended_reactions_0)


### Create the graphs from the 2 conditions (subset network, with reactions from the 2 pathways)

In [23]:

subset_G100, subset_pos100 = construct_graph(subset_linear_correlation_matrix_100,
                                             subset_non_linear_correlation_matrix_100,
                                             reactions=subset_extended_reactions_100,
                                             remove_unconnected_nodes=False,
                                             correction=False,
                                             group_map=group_map_100)


subset_G0, subset_pos0 = construct_graph(subset_linear_correlation_matrix_0, 
                                         subset_non_linear_correlation_matrix_0, 
                                         reactions=subset_extended_reactions_0,
                                         remove_unconnected_nodes=False,
                                         correction=False,
                                         group_map=group_map_0)


### Plot the graphs from the subset networks of the 2 conditions

In [25]:

plot_graph(subset_G100, 
           subset_pos100, 
           remove_clique_edges=True, 
           include_matrix2_in_cliques=False, 
           min_clique_size=5, 
           shadow_edges="mixed", 
           centralities=centrality_dict_100)


plot_graph(subset_G0, 
           subset_pos0, 
           remove_clique_edges=True, 
           include_matrix2_in_cliques=False, 
           min_clique_size=5, 
           shadow_edges="mixed", 
           centralities=centrality_dict_0)
